In [1]:
import pandas as pd
import numpy as np

data_file_name = 'data.csv'
labels_file_name = 'what_answer_codes_mean.csv'
questions_file_name = 'what_questsions_are.csv'

data = pd.read_csv(data_file_name)
labels = pd.read_csv(labels_file_name)
questions = pd.read_csv(questions_file_name)

In [2]:
# remove data where country is Nan
data = data[data['Residency'] != 'Nan']
data = data.dropna()
data = data[data['var'] == data['var'].astype(str)]

# remove data in what_questions_are where "question" is not a string
questions = questions[questions['question'] == questions['question'].astype(str)]
# get and remove the question data where [var] is NA
group_header_statements = questions[questions['row_id'] == 0]
questions = questions[questions['row_id'] != 0]

# get unique countries
countries = data['Residency'].unique()
# get unique questions
unique_questions = data['var'].unique()

In [3]:
# create new csv file
columns = np.insert(unique_questions,0,"rowid")
new_data = pd.DataFrame(columns=columns)


In [4]:
# for each entry in data, add it to the new csv file
person_dict = {}
for index, row in data.iterrows():
    # add the row to the new csv file
    person_dict[row['rowid']] = person_dict.get(row['rowid'], {})
    person_dict[row['rowid']][row['var']] = row['code']

In [5]:
for key in person_dict:
    new_data.loc[len(new_data.index)] = person_dict[key]

In [6]:
# save the new csv file
new_data.to_csv('person_dat.csv', index=False)

In [9]:
country_dict = {}

for index, row in data.iterrows():
    country_dict[row['Residency']] = country_dict.get(row['Residency'], {})
    country_dict[row['Residency']][row['var']] = country_dict[row['Residency']].get(row['var'], [])
    country_dict[row['Residency']]['DemGen'] = country_dict[row['Residency']].get('DemGen', [])
    country_dict[row['Residency']]['quota_age'] = country_dict[row['Residency']].get('quota_age', [])
    country_dict[row['Residency']][row['var']].append(row['code'])
    country_dict[row['Residency']]['DemGen'].append(row['DemGen'])
    country_dict[row['Residency']]['quota_age'].append(row['quota_age'])
    country_dict[row['Residency']]['EndDate'] = country_dict[row['Residency']].get('EndDate', [])
    country_dict[row['Residency']]['EndDate'].append(row['EndDate'][0:10] + ' 00:00:00')

In [28]:
for key in country_dict.keys():
    for key2 in country_dict[key].keys():
        if key2 == 'EndDate':
            dates = country_dict[key][key2]
            mean = (np.array(dates, dtype='datetime64[s]')
            .view('i8')
            .mean()
            .astype('datetime64[s]'))
            mean = str(mean)[0:10]
            country_dict[key][key2] = mean
            print(mean)

2020-03-22
2020-04-03
2020-03-19
2020-03-22
2020-03-23
2020-03-22
2020-03-22
2020-03-28
2020-04-10
2020-04-09
2020-04-09


In [30]:
df = pd.DataFrame.from_dict(country_dict, orient='index')
# df.to_json('data_with_dates.json')

In [ ]:
df = pd.read_json('data.json')
# print(df)
# print column names
print(df['GenSocTrust'])

In [8]:
names = pd.read_csv('country_code_data.csv')
print(names)

                  Name Code
0          Afghanistan   AF
1        Åland Islands   AX
2              Albania   AL
3              Algeria   DZ
4       American Samoa   AS
..                 ...  ...
244  Wallis and Futuna   WF
245     Western Sahara   EH
246              Yemen   YE
247             Zambia   ZM
248           Zimbabwe   ZW

[249 rows x 2 columns]


In [ ]:
country_dict2 = {}
country_names = {
    'IT': 'Italy',
    'UK': 'United Kingdom',
    'US': 'United States',
    'AU': 'Australia',
    'DE': 'Germany',
    'ES': 'Spain',
    'MX': 'Mexico',
    'SE': 'Sweden',
    'JP': 'Japan',
    'KR': 'South Korea',
    'CN': 'China'
}
for key in country_dict.keys():
    name = country_names[key]
    country_dict2[name] = country_dict2.get(name, {})
    for key2 in country_dict[key].keys():
        try:
            values = country_dict[key][key2]
            new_values = []
            for value in values:
                if type(value) == str:
                    new_value = value.split(",")
                    new_values.extend(new_value)
            values = [int(i) for i in values]
            mean = np.mean(values)
            country_dict2[name] = country_dict2.get(name, {})
            country_dict2[name][key2] = mean
        except:
            country_dict2[name] = country_dict2.get(name, {})
            country_dict2[name][key2] = country_dict[key][key2]

print(country_dict2)

In [ ]:
stringency_data = pd.read_csv('covid-stringency-index.csv')
for key in country_dict2.keys():
    for index, row in stringency_data.iterrows():
        if row['Entity'] == key and row['Day'] == country_dict2[key]['EndDate']:
            country_dict2[key]['stringnecy'] = row['stringency_index']
            break
print(country_dict2)

In [41]:
df = pd.DataFrame.from_dict(country_dict2, orient='index')
df.to_json('mean_data_per_country.json')